In [ ]:
import bs4, re
from bs4 import BeautifulSoup as soup
import requests

URL = "https://www.musicgenreslist.com/"
page = requests.get(URL)
html = page.content
s = soup(html, "html.parser")

In [ ]:
# FIND ROOT ELEMENT
root_element = None
alla = s.findAll('a')
for a in alla:
    if a.attrs.get('name') ==  "music-genre-list":
        #print(a.parent.parent)
        root_element = a.parent.parent.find('div')


In [ ]:
# filter unwanted stuff
root_list = list(root_element)
tentative_lists = list(filter(lambda x: (x != '\n'), root_list)) 

print(len(tentative_lists[0]))
print(len(tentative_lists[1]))
l0 = tentative_lists[0]
l1 = tentative_lists[1]
ll = list(l0) + list(l1)

In [192]:
# FILTER
tenative_dict = {}
for item in ll:
    #print("GENRRE")
    a = item.find('a')
    if a == -1 or a is None:
        continue
    
    title = a.attrs.get('title')
    if title is None:
        title = a.attrs.get('href')
    tenative_dict[title] = item

In [274]:
def make_key(item):
    key = item.text.encode("ascii", "ignore").decode()
    key = key.replace('\n','')
    return key

def rec_parse(item):
    if len(item) == 1:
        return make_key(item),None
    else:
        l = {}
        key = make_key(list(item)[0])
        for sub_item in item.findAll('li'):
            kk, vv =  rec_parse(sub_item)
            #print(kk,vv)
            l[kk] = vv 
        return key,l

tentative_dict_v2 = {}
for genre, item in tenative_dict.items():
    if genre.endswith(' - Music Genre'):
        genre = genre[0:-len(" - Music Genre")]
    elif genre == "https://www.musicgenreslist.com/metal-music-genre/":
        genre = "Metal"
    sub = item.find('ul')
    if sub is None:
        continue
    
    subgenres = {}
    for sub_item in item.findAll('li'):
        key, value = rec_parse(sub_item)
        subgenres[key] = value
            
        
        
    #li = [sub_item.text for sub_item in sub.findAll('li')]
    #print(f"{genre} #subgenre {len(li)}")
    tentative_dict_v2[genre] = subgenres

In [280]:
tentative_dict_v2
tentative_dict_v2['__meta__'] = {
    "credit":[
        "the intial version of this list was parsed from https://www.musicgenreslist.com",
    ]
}
import json
with open("assets/json/genres_v0.json", "w") as f:
    json.dump(tentative_dict_v2,f, indent=4)